# DES Y6 Deep Field Exposures:  Photometric Zeropoints tied to ATLAS-REFCAT2

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import glob
import math
import os
import subprocess
import sys
import gc

import glob
import pickle

import easyaccess as ea

#import AlasBabylon

import fitsio
from astropy.io import fits
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack

import tempfile

import matplotlib.pyplot as plt
 
%matplotlib inline

In [2]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. User Input

### 2.1.  General User Input  

In [3]:
verbose = 1
tag_des = 'Y6A2_FINALCUT'      # Official tag for DES Y6A2_FINALCUT
tag_decade = 'DECADE_FINALCUT' # Tag for DECADE
rawdata_dir = '../RawData'
zeropoints_dir='../Zeropoints'

bandList = ['g', 'r', 'i', 'z', 'Y']

### 2.2. Logical Variables to Indicate which Code Cells to Run

In [4]:
do_calc_refcat2_zps = True

### 2.3. Sky Region Definitions

In [5]:
region_name_list = [
                    'VVDSF14', 
                    'VVDSF22', 
                    'DEEP2', 
                    'SN-E', 
                    'SN-X_err', 
                    'SN-X', 
                    'ALHAMBRA2', 
                    'SN-S',
                    'SN-C', 
                    'EDFS',
                    'MACS0416',
                    'SN-S_err',
                    'COSMOS'
                    ]

region_ramin = {
                'VVDSF14':208.,
                'VVDSF22':333.,
                'DEEP2':351., 
                'SN-E':6., 
                'SN-X_err':13., 
                'SN-X':32., 
                'ALHAMBRA2':35., 
                'SN-S':39.5,
                'SN-C':50., 
                'EDFS':55.,
                'MACS0416':62.,
                'SN-S_err':83.5,
                'COSMOS':148.
               }

region_ramax = {
                'VVDSF14':212.,
                'VVDSF22':337.,
                'DEEP2':354., 
                'SN-E':12., 
                'SN-X_err':17., 
                'SN-X':38., 
                'ALHAMBRA2':39., 
                'SN-S':44.5,
                'SN-C':56., 
                'EDFS':67.,
                'MACS0416':66.,
                'SN-S_err':88.,
                'COSMOS':153.
               }

region_decmin = {
                'VVDSF14':3.,
                'VVDSF22':-1.5,
                'DEEP2':-1.5, 
                'SN-E':-46., 
                'SN-X_err':-32., 
                'SN-X':-8., 
                'ALHAMBRA2':-0.5, 
                'SN-S':-2.5,
                'SN-C':-31., 
                'EDFS':-51.,
                'MACS0416':-25.5,
                'SN-S_err':-38.,
                'COSMOS':0.5
               }


region_decmax = {
                'VVDSF14':7.,
                'VVDSF22':1.5,
                'DEEP2':1.5, 
                'SN-E':-41., 
                'SN-X_err':-29., 
                'SN-X':-3., 
                'ALHAMBRA2':2.5, 
                'SN-S':1.5,
                'SN-C':-25., 
                'EDFS':-46.,
                'MACS0416':-22.5,
                'SN-S_err':-35.,
                'COSMOS':4.0
               }


for regionName in region_name_list:
    print regionName, region_ramin[regionName], region_ramax[regionName], region_decmin[regionName], region_decmax[regionName]

VVDSF14 208.0 212.0 3.0 7.0
VVDSF22 333.0 337.0 -1.5 1.5
DEEP2 351.0 354.0 -1.5 1.5
SN-E 6.0 12.0 -46.0 -41.0
SN-X_err 13.0 17.0 -32.0 -29.0
SN-X 32.0 38.0 -8.0 -3.0
ALHAMBRA2 35.0 39.0 -0.5 2.5
SN-S 39.5 44.5 -2.5 1.5
SN-C 50.0 56.0 -31.0 -25.0
EDFS 55.0 67.0 -51.0 -46.0
MACS0416 62.0 66.0 -25.5 -22.5
SN-S_err 83.5 88.0 -38.0 -35.0
COSMOS 148.0 153.0 0.5 4.0


### 2.4. Check on Location of TMPDIR...

In [6]:
# Check on TMPDIR... 
tempfile.gettempdir()

'/data/des40.a/data/dtucker/TmpDir'

In [7]:
# Set tmpdir variable to $TMPDIR (for future reference)...
tmpdir = os.environ['TMPDIR']

### 2.5. Create Main Zeropoints Directory (if it does not already exist)...

In [8]:
# Create main Zeropoints directory, if it does not already exist...
if not os.path.exists(zeropoints_dir):
    os.makedirs(zeropoints_dir)

## 3. Useful Modules

In [9]:
def DECam_tie_to_refcat2(inputFile, outputFile, band, fluxObsColName, fluxerrObsColName, aggFieldColName, verbose):

    import numpy as np 
    import os
    import sys
    import datetime
    import pandas as pd
    from astropy.table import Table, vstack
    
    validBandsList = ['g', 'r', 'i', 'z', 'Y']

    if band not in validBandsList:
        print """Filter band %s is not currently handled...  Exiting now!""" % (band)
        return 1
    
    reqColList = ['g','r','i','z','dg','dr','di','dz',
                  fluxObsColName,fluxerrObsColName,aggFieldColName]        
      
    # Does the input file exist?
    if os.path.isfile(inputFile)==False:
        print """DECam_tie_to_refcat2 input file %s does not exist.  Exiting...""" % (inputFile)
        return 1

    # Read inputFile into a pandas DataFrame...
    print datetime.datetime.now()
    print """Reading in %s as a pandas DataFrame...""" % (inputFile)
    t = Table.read(inputFile)
    dataFrame = t.to_pandas()
    print datetime.datetime.now()
    print

    reqColFlag = 0
    colList = dataFrame.columns.tolist()
    for reqCol in reqColList:
        if reqCol not in colList:
            print """ERROR:  Required column %s is not in the header""" % (reqCol)
            reqColFlag = 1
    if reqColFlag == 1:
        print """Missing required columns in header of %s...  Exiting now!""" (inputFile)
        return 1

    # Identify column of the standard magnitude for the given band...
    magStdColName = """MAG_STD_%s""" % (band.upper())
    magerrStdColName = """MAGERR_STD_%s""" % (band.upper())
    
    # Transform ATLAS-REFCAT2 mags to DES mags for the given band...
    dataFrame = transREFCAT2toDES(dataFrame, band, magStdColName, magerrStdColName)

    # Add a 'MAG_OBS' column and a 'MAG_DIFF' column to the pandas DataFrame...
    dataFrame['MAG_OBS'] = -2.5*np.log10(dataFrame[fluxObsColName])
    dataFrame['MAG_DIFF'] = dataFrame[magStdColName]-dataFrame['MAG_OBS']


    ###############################################
    # Aggregate by aggFieldColName
    ###############################################

    # Make a copy of original dataFrame...
    df = dataFrame.copy()

    # Create an initial mask...
    mask1 = ( (df[magStdColName] >= 0.) & (df[magStdColName] <= 25.) )
    mask1 = ( mask1 & (df[fluxObsColName] > 10.) & (df['FLAGS'] < 2) & (np.abs(df['SPREAD_MODEL']) < 0.01))
    if magerrStdColName != 'None':  
        mask1 = ( mask1 & (df[magerrStdColName] < 0.1) )
    magDiffGlobalMedian = df[mask1]['MAG_DIFF'].median()
    magDiffMin = magDiffGlobalMedian - 5.0
    magDiffMax = magDiffGlobalMedian + 5.0
    mask2 = ( (df['MAG_DIFF'] > magDiffMin) & (df['MAG_DIFF'] < magDiffMax) )
    mask = mask1 & mask2

    # Iterate over the copy of dataFrame 3 times, removing outliers...
    #  We are using "Method 2/Group by item" from
    #  http://nbviewer.jupyter.org/urls/bitbucket.org/hrojas/learn-pandas/raw/master/lessons/07%20-%20Lesson.ipynb
    print "Sigma-clipping..."
    niter = 0
    for i in range(3):

        niter = i + 1
        print """   iter%d...""" % ( niter )

        # make a copy of original df, and then delete the old one...
        newdf = df[mask].copy()
        del df
        # group by aggFieldColName...
        grpnewdf = newdf.groupby([aggFieldColName])
        
        # add/update new columns to newdf
        print datetime.datetime.now()
        newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 3.00*x.std() )
        #newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 2.00*x.std() )
        print datetime.datetime.now()
        del grpnewdf
        print datetime.datetime.now()
        #print newdf

        nrows = newdf['MAG_DIFF'].size
        print """  Number of rows remaining:  %d""" % ( nrows )

        df = newdf
        mask = ( df['Outlier'] == False )  


    # Perform pandas grouping/aggregating functions on sigma-clipped Data Frame...
    print datetime.datetime.now()
    print 'Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...'
    groupedDataFrame = df.groupby([aggFieldColName])
    magZeroMedian = groupedDataFrame['MAG_DIFF'].median()
    magZeroMean = groupedDataFrame['MAG_DIFF'].mean()
    magZeroStd = groupedDataFrame['MAG_DIFF'].std()
    magZeroNum = groupedDataFrame['MAG_DIFF'].count()
    magZeroErr = magZeroStd/np.sqrt(magZeroNum-1)
    print datetime.datetime.now()
    print

    # Rename these pandas series...
    magZeroMedian.name = 'MAG_ZERO_MEDIAN'
    magZeroMean.name = 'MAG_ZERO_MEAN'
    magZeroStd.name = 'MAG_ZERO_STD'
    magZeroNum.name = 'MAG_ZERO_NUM'
    magZeroErr.name = 'MAG_ZERO_MEAN_ERR'

    # Also, calculate group medians for all columns in df that have a numerical data type...
    numericalColList = df.select_dtypes(include=[np.number]).columns.tolist()
    groupedDataMedian = {}
    for numericalCol in numericalColList:
        groupedDataMedian[numericalCol] = groupedDataFrame[numericalCol].median()
        groupedDataMedian[numericalCol].name = """%s_MEDIAN""" % (numericalCol)

    # Create new data frame containing all the relevant aggregate quantities
    #newDataFrame = pd.concat( [magZeroMedian, magZeroMean, magZeroStd, \
    #                           magZeroErr, magZeroNum], \
    #                           join='outer', axis=1 )
    seriesList = []
    for numericalCol in numericalColList:
        seriesList.append(groupedDataMedian[numericalCol])
    seriesList.extend([magZeroMedian, magZeroMean, magZeroStd, \
                               magZeroErr, magZeroNum])
    #print seriesList
    newDataFrame = pd.concat( seriesList, join='outer', axis=1 )
                               

    #newDataFrame.index.rename('FILENAME', inplace=True)

    # Saving catname-based results to output files...
    print datetime.datetime.now()
    print """Writing %s output file (using pandas to_csv method)...""" % (outputFile)
    newDataFrame.to_csv(outputFile, float_format='%.4f')
    print datetime.datetime.now()
    print

    return 0

In [12]:
# Transform ATLAS-REFCAT2 mags into DES mags for this filter band...
def transREFCAT2toDES(dataFrame, band, magStdColName, magerrStdColName):

    import numpy as np 
    import pandas as pd
    from collections import OrderedDict as odict

    # Transformation coefficients (updated based on fit to DES).
    # mag_des = mag_refcat2 + A[mag][0]*color_refcat2 + A[mag][1]
    # (from A. Drlica-Wagner's https://github.com/kadrlica/desqr/blob/master/desqr/calibrate.py)
    REFCAT2 = odict([
            ('g', [+0.0994, -0.0076 - 0.0243]), # [-0.2 < (g-r)_refcat2 <= 1.2]
            ('r', [-0.1335, +0.0189 + 0.0026]), # [-0.2 < (g-r)_refcat2 <= 1.2]
            ('i', [-0.3407, +0.0026 - 0.0039]), # [-0.2 < (i-z)_refcat2 <= 0.3]
            ('z', [-0.2575, -0.0074 - 0.0127]), # [-0.2 < (i-z)_refcat2 <= 0.3]
            ('Y', [-0.6032, +0.0185]),          # [-0.2 < (i-z)_refcat2 <= 0.3]
            ]) 
    
    A = REFCAT2

    if band is 'g':
        # g_des = g_refcat2 + 0.0994*(g-r)_refcat2 - 0.0076 - 0.0243   [-0.2 < (g-r)_ps <= 1.2]
        dataFrame[magStdColName] = dataFrame['g']+\
            A[band][0]*(dataFrame['g']-dataFrame['r'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['dg']  # temporary
        mask  = ( (dataFrame['g']-dataFrame['r']) > -0.2)
        mask &= ( (dataFrame['g']-dataFrame['r']) <= 1.2)
    elif band is 'r':
        # r_des = r_refcat2 - 0.1335*(g-r)_refcat2 + 0.0189 + 0.0026   [-0.2 < (g-r)_ps <= 1.2]
        dataFrame[magStdColName] = dataFrame['r']+\
            A[band][0]*(dataFrame['g']-dataFrame['r'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['dr']  # temporary
        mask  = ( (dataFrame['g']-dataFrame['r']) > -0.2)
        mask &= ( (dataFrame['g']-dataFrame['r']) <= 1.2)
    elif band is 'i':
        # i_des = i_refcat2 - 0.3407*(i-z)_refcat2 + 0.0026 - 0.0039   [-0.2 < (i-z)_ps <= 0.3]
        dataFrame[magStdColName] = dataFrame['i']+\
            A[band][0]*(dataFrame['i']-dataFrame['z'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['di']  # temporary
        mask  = ( (dataFrame['i']-dataFrame['z']) > -0.2)
        mask &= ( (dataFrame['i']-dataFrame['z']) <= 0.3)
    elif band is 'z':
        # z_des = z_refcat2 - 0.2575*(i-z)_refcat2 - 0.0074 - 0.0127   [-0.2 < (i-z)_ps <= 0.3]
        dataFrame[magStdColName] = dataFrame['z']+\
            A[band][0]*(dataFrame['i']-dataFrame['z'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['dz']  # temporary
        mask  = ( (dataFrame['i']-dataFrame['z']) > -0.2)
        mask &= ( (dataFrame['i']-dataFrame['z']) <= 0.3)
    elif band is 'Y':
        # Y_des = z_refcat2 - 0.6032*(i-z)_refcat2 + 0.0185    [-0.2 < (i-z)_ps <= 0.3]
        dataFrame[magStdColName] = dataFrame['z']+\
            A[band][0]*(dataFrame['i']-dataFrame['z'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['dz']  # temporary
        mask  = ( (dataFrame['i']-dataFrame['z']) > -0.2)
        mask &= ( (dataFrame['i']-dataFrame['z']) <= 0.3)
    else:
        msg = "Unrecognized band: %s "%band
        raise ValueError(msg)

    dataFrame = dataFrame[mask].copy()

    return dataFrame

## 4. Zeropoints by tying to DES-transformed ATLAS-REFCAT2 Stars

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

Calculate zeropoints region by region...

In [13]:
%%time 

if do_calc_refcat2_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.refcat2.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_refcat2(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.VVDSF14.0.g.refcat2.csv
2022-01-04 16:20:40.984757
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.refcat2.fits as a pandas DataFrame...
2022-01-04 16:20:42.045341

Sigma-clipping...
   iter1...
2022-01-04 16:20:42.167411
2022-01-04 16:20:42.461062
2022-01-04 16:20:42.461326
  Number of rows remaining:  15251
   iter2...
2022-01-04 16:20:42.485701
2022-01-04 16:20:42.776746
2022-01-04 16:20:42.776957
  Number of rows remaining:  14882
   iter3...
2022-01-04 16:20:42.794226
2022-01-04 16:20:43.085456
2022-01-04 16:20:43.085663
  Number of rows remaining:  14636
2022-01-04 16:20:43.086534
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 16:20:43.093880

2022-01-04 16:20:43.205962
Writing ../Zeropoints/zp

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.DEEP2.?.Y.refcat2.fits

# # # # # # # # # # # # # # #
Working on region SN-E
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.g.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.g.refcat2.csv
2022-01-04 16:20:52.442682
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.g.refcat2.fits as a pandas DataFrame...
2022-01-04 16:21:11.085683

Sigma-clipping...
   iter1...
2022-01-04 16:21:14.946864
2022-01-04 16:21:22.800315
2022-01-04 16:21:22.803758
  Number of rows remaining:  153650
   iter2...
2022-01-04 16:21:23.617816
2022-01-04 16:21:31.472803
2022-01-04 16:21:31.477185
  Number of rows remaining:  150370
   iter3...
2022-01-04 16:21:31.931462
2022-01-04 16:21:39.763465
2022-01-04 16:21:39.766541
  Number of rows remaining:  149578
2022-01-04 16:21:39.768261
Performing grouping/aggregating functions on sigma-clip

2022-01-04 16:31:08.754432

Sigma-clipping...
   iter1...
2022-01-04 16:31:14.401125
2022-01-04 16:31:33.846933
2022-01-04 16:31:33.855757
  Number of rows remaining:  274097
   iter2...
2022-01-04 16:31:35.550758
2022-01-04 16:31:55.524729
2022-01-04 16:31:55.532901
  Number of rows remaining:  270854
   iter3...
2022-01-04 16:31:56.554392
2022-01-04 16:32:16.456017
2022-01-04 16:32:16.464078
  Number of rows remaining:  270621
2022-01-04 16:32:16.467093
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 16:32:16.591509

2022-01-04 16:32:18.979924
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.0.i.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 16:32:26.887533


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.1.i.refcat2.csv
2022-01-04 16:32:27.046750
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.refcat2.fits as a 

2022-01-04 16:42:33.800624

Sigma-clipping...
   iter1...
2022-01-04 16:42:33.950641
2022-01-04 16:42:35.064662
2022-01-04 16:42:35.065391
  Number of rows remaining:  15211
   iter2...
2022-01-04 16:42:35.105448
2022-01-04 16:42:36.208180
2022-01-04 16:42:36.210333
  Number of rows remaining:  15047
   iter3...
2022-01-04 16:42:36.258389
2022-01-04 16:42:37.386541
2022-01-04 16:42:37.388014
  Number of rows remaining:  15034
2022-01-04 16:42:37.389471
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 16:42:37.399271

2022-01-04 16:42:37.538840
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.i.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 16:42:37.962969


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.refcat2.csv
2022-01-04 16:42:37.978980
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.r

2022-01-04 16:53:30.958426


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.i.refcat2.csv
2022-01-04 16:53:31.416543
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.refcat2.fits as a pandas DataFrame...
2022-01-04 16:54:15.541919

Sigma-clipping...
   iter1...
2022-01-04 16:54:23.574118
2022-01-04 16:54:47.817477
2022-01-04 16:54:47.843394
  Number of rows remaining:  365036
   iter2...
2022-01-04 16:54:50.750810
2022-01-04 16:55:14.736407
2022-01-04 16:55:14.752624
  Number of rows remaining:  361881
   iter3...
2022-01-04 16:55:16.521690
2022-01-04 16:55:40.253313
2022-01-04 16:55:40.267835
  Number of rows remaining:  361609
2022-01-04 16:55:40.271757
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 16:55:40.477320

2022-01-04 16:55:43.369032
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.i.refcat2.csv output file (using panda

Sigma-clipping...
   iter1...
2022-01-04 17:08:23.627550
2022-01-04 17:08:24.920077
2022-01-04 17:08:24.920749
  Number of rows remaining:  46708
   iter2...
2022-01-04 17:08:25.031711
2022-01-04 17:08:26.325756
2022-01-04 17:08:26.326420
  Number of rows remaining:  45378
   iter3...
2022-01-04 17:08:26.395840
2022-01-04 17:08:27.672867
2022-01-04 17:08:27.673532
  Number of rows remaining:  44474
2022-01-04 17:08:27.674746
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 17:08:27.691171

2022-01-04 17:08:28.000242
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.g.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 17:08:28.492619


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.r.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.r.refcat2.csv
2022-01-04 17:08:28.511869
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.r.refcat2.fits as a pandas DataFram

2022-01-04 17:10:03.487375
2022-01-04 17:10:03.490860
  Number of rows remaining:  84884
2022-01-04 17:10:03.492951
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 17:10:03.527087

2022-01-04 17:10:04.067966
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.i.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 17:10:05.513194


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.refcat2.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.z.refcat2.csv
2022-01-04 17:10:05.554417
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.refcat2.fits as a pandas DataFrame...
2022-01-04 17:10:26.069710

Sigma-clipping...
   iter1...
2022-01-04 17:10:27.824907
2022-01-04 17:10:35.471290
2022-01-04 17:10:35.475203
  Number of rows remaining:  228292
   iter2...
2022-01-04 17:10:35.969332
2022-01-04 17:10:43.608865
2022-01-04 17:10:43.612399
  Number of rows remaining:  225371
   iter3...


Combine region-by-region results into a single file...

In [15]:
%%time 

if do_calc_refcat2_zps: 
        
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.refcat2.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.refcat2.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.refcat2.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.g.refcat2.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.r.refcat2.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.i.refcat2.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.z.refcat2.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DES_Y6A2_FINALCUT.*.?.Y.refcat2.csv
CPU times: user 1min 33s, sys: 3.38 s, total: 1min 36s
Wall time: 1min 36s


**DECADE:**

Calculate zeropoints region by region...

In [16]:
%%time 

if do_calc_refcat2_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.refcat2.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_refcat2(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.g.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.r.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.i.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.z.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.Y.refcat2.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.refcat2.fits
../Zeropoints/zps_DECADE_FINALCUT.VVDSF22.0.g.refcat2.csv
2022-01-04 17:29:25.262615
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.refcat2.fits as a pandas 

2022-01-04 17:33:52.690407
2022-01-04 17:33:59.514936
2022-01-04 17:33:59.519578
  Number of rows remaining:  79112
   iter3...
2022-01-04 17:33:59.733961
2022-01-04 17:34:06.574010
2022-01-04 17:34:06.578754
  Number of rows remaining:  79079
2022-01-04 17:34:06.581221
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 17:34:06.620311

2022-01-04 17:34:07.234000
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-E.0.z.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 17:34:10.005022


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-E.?.Y.refcat2.fits

# # # # # # # # # # # # # # #
Working on region SN-X_err
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.g.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.r.refcat2.fits
No files matching template ../RawData/Exp

2022-01-04 17:36:09.070819


../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.refcat2.fits
../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.i.refcat2.csv
2022-01-04 17:36:09.089778
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.refcat2.fits as a pandas DataFrame...
2022-01-04 17:36:13.913002

Sigma-clipping...
   iter1...
2022-01-04 17:36:14.446867
2022-01-04 17:36:17.045748
2022-01-04 17:36:17.047708
  Number of rows remaining:  43105
   iter2...
2022-01-04 17:36:17.184622
2022-01-04 17:36:19.790834
2022-01-04 17:36:19.791963
  Number of rows remaining:  42527
   iter3...
2022-01-04 17:36:19.879672
2022-01-04 17:36:22.498921
2022-01-04 17:36:22.501480
  Number of rows remaining:  42445
2022-01-04 17:36:22.503067
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 17:36:22.528375

2022-01-04 17:36:22.846632
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.i.refcat2.csv output file (using pandas to_csv method

2022-01-04 17:40:39.344991
2022-01-04 17:40:39.354190
  Number of rows remaining:  37429
2022-01-04 17:40:39.357442
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 17:40:39.414753

2022-01-04 17:40:39.908557
Writing ../Zeropoints/zps_DECADE_FINALCUT.EDFS.0.z.refcat2.csv output file (using pandas to_csv method)...
2022-01-04 17:40:43.124125


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.?.Y.refcat2.fits

# # # # # # # # # # # # # # #
Working on region MACS0416
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.MACS0416.?.g.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.MACS0416.?.r.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.MACS0416.?.i.refcat2.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.

Combine region-by-region results into a single file...

In [17]:
%%time 

if do_calc_refcat2_zps: 
        
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.refcat2.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.refcat2.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.refcat2.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.g.refcat2.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.r.refcat2.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.i.refcat2.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.z.refcat2.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DECADE_FINALCUT.*.?.Y.refcat2.csv
CPU times: user 20 s, sys: 922 ms, total: 20.9 s
Wall time: 21 s
